In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1317.39,-0.01,0.01,0.02,0.00,0.00,0.02,0.03,-0.00,0.01,...,0.00,0.0,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00
3,-0.01,935.98,0.00,-0.02,0.02,-0.03,0.00,0.01,-0.00,-0.00,...,-0.00,0.0,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00
4,0.01,0.00,720.17,0.03,0.01,-0.00,0.01,-0.01,0.01,0.00,...,-0.00,0.0,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00
5,0.02,-0.02,0.03,529.15,-0.02,0.02,-0.00,-0.01,0.00,0.00,...,-0.00,-0.0,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00
6,0.00,0.02,0.01,-0.02,363.76,-0.00,-0.00,-0.00,-0.01,0.00,...,0.00,-0.0,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00
7,0.00,-0.03,-0.00,0.02,-0.00,232.21,-0.00,0.00,0.00,-0.00,...,-0.00,0.0,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00
8,0.02,0.00,0.01,-0.00,-0.00,-0.00,152.61,0.00,-0.01,0.00,...,0.00,0.0,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00
9,0.03,0.01,-0.01,-0.01,-0.00,0.00,0.00,91.63,-0.00,-0.00,...,0.00,-0.0,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
10,-0.00,-0.00,0.01,0.00,-0.01,0.00,-0.01,-0.00,59.51,0.00,...,0.00,-0.0,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00
11,0.01,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,38.10,...,0.00,0.0,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1.00000,-0.00001,0.00002,0.00003,0.00000,0.00001,0.00004,0.00010,-0.00001,0.00003,...,0.00001,0.00001,-0.00002,0.00007,0.00005,-0.00000,0.00002,-0.00001,0.00003,-0.00002
3,-0.00001,1.00000,0.00000,-0.00002,0.00003,-0.00006,0.00001,0.00003,-0.00002,-0.00002,...,-0.00001,0.00004,0.00000,0.00003,0.00003,-0.00001,0.00003,-0.00004,-0.00004,-0.00001
4,0.00002,0.00000,1.00000,0.00004,0.00002,-0.00000,0.00003,-0.00004,0.00007,0.00000,...,-0.00004,0.00001,0.00003,-0.00004,0.00000,0.00001,0.00000,-0.00002,0.00002,0.00000
5,0.00003,-0.00002,0.00004,1.00000,-0.00004,0.00005,-0.00001,-0.00003,0.00001,0.00001,...,-0.00002,-0.00002,0.00001,-0.00001,0.00004,0.00000,0.00000,-0.00002,0.00005,0.00003
6,0.00000,0.00003,0.00002,-0.00004,1.00000,-0.00000,-0.00000,-0.00000,-0.00004,0.00003,...,0.00002,-0.00001,0.00002,0.00000,0.00003,0.00004,-0.00002,0.00002,-0.00004,-0.00001
7,0.00001,-0.00006,-0.00000,0.00005,-0.00000,1.00000,-0.00000,0.00000,0.00003,-0.00002,...,-0.00004,0.00002,-0.00003,0.00008,-0.00003,0.00005,0.00001,-0.00005,0.00000,-0.00002
8,0.00004,0.00001,0.00003,-0.00001,-0.00000,-0.00000,1.00000,0.00001,-0.00005,0.00005,...,0.00001,0.00003,-0.00001,0.00000,-0.00000,0.00000,-0.00002,-0.00007,-0.00003,-0.00000
9,0.00010,0.00003,-0.00004,-0.00003,-0.00000,0.00000,0.00001,1.00000,-0.00003,-0.00002,...,0.00001,-0.00000,-0.00001,0.00005,-0.00002,0.00003,0.00001,0.00002,-0.00003,-0.00006
10,-0.00001,-0.00002,0.00007,0.00001,-0.00004,0.00003,-0.00005,-0.00003,1.00000,0.00006,...,0.00001,-0.00001,0.00000,-0.00002,0.00001,0.00003,-0.00003,0.00001,0.00001,0.00002
11,0.00003,-0.00002,0.00000,0.00001,0.00003,-0.00002,0.00005,-0.00002,0.00006,1.00000,...,0.00002,0.00001,0.00005,-0.00001,0.00004,0.00006,-0.00004,-0.00001,-0.00006,0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.11272672550858809

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.57192267e+04 3.81254662e+04 2.24616729e+04 1.20823428e+04
 5.69339884e+03 2.31575552e+03 9.95907814e+02 3.58392195e+02
 1.50404372e+02 6.13993337e+01 2.64625798e+01 1.16854827e+01
 5.20599363e+00 2.12771032e+00 8.54665632e-01 3.03759346e-01
 1.04831849e-01 2.74219289e-02 7.71188881e-03 1.88553081e-03
 7.83297435e-04 3.99018882e-04 1.30038679e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.997164,-0.062543,-0.033093,-0.020110,-0.012510,-0.007594,-0.004887,-0.002888,-0.001890,-0.001202,...,-0.000227,-0.000145,-0.000088,-0.000050,-0.000026,-0.000015,-0.000007,-0.000005,-0.000003,-0.000003
1,0.058185,0.993295,-0.089054,-0.037530,-0.020353,-0.011749,-0.007403,-0.004355,-0.002827,-0.001805,...,-0.000340,-0.000214,-0.000130,-0.000076,-0.000039,-0.000022,-0.000010,-0.000007,-0.000006,-0.000004
2,0.035769,0.082633,0.991627,-0.083166,-0.033863,-0.017796,-0.010908,-0.006375,-0.004069,-0.002608,...,-0.000496,-0.000313,-0.000186,-0.000114,-0.000058,-0.000031,-0.000015,-0.000010,-0.000007,-0.000005
3,0.023833,0.040869,0.075701,0.992709,-0.072655,-0.029111,-0.016751,-0.009489,-0.006033,-0.003836,...,-0.000724,-0.000463,-0.000275,-0.000165,-0.000083,-0.000046,-0.000022,-0.000015,-0.000010,-0.000008
4,0.015791,0.024040,0.035314,0.066127,0.994177,-0.062845,-0.028896,-0.015141,-0.009457,-0.005929,...,-0.001109,-0.000710,-0.000423,-0.000253,-0.000129,-0.000069,-0.000035,-0.000022,-0.000017,-0.000013
5,0.009967,0.014496,0.019630,0.029389,0.056989,0.994519,-0.070343,-0.027450,-0.015982,-0.009823,...,-0.001813,-0.001149,-0.000694,-0.000404,-0.000213,-0.000113,-0.000056,-0.000038,-0.000027,-0.000021
6,0.006897,0.009846,0.013010,0.018428,0.029616,0.065201,0.994575,-0.060928,-0.029170,-0.016703,...,-0.002962,-0.001882,-0.001135,-0.000673,-0.000348,-0.000188,-0.000094,-0.000062,-0.000045,-0.000033
7,0.004209,0.005983,0.007855,0.010821,0.016249,0.028032,0.055042,0.994065,-0.076246,-0.032945,...,-0.005111,-0.003250,-0.001951,-0.001151,-0.000600,-0.000321,-0.000159,-0.000102,-0.000078,-0.000059
8,0.002948,0.004161,0.005396,0.007409,0.010949,0.017763,0.029659,0.069876,0.992627,-0.079382,...,-0.008573,-0.005422,-0.003242,-0.001926,-0.000992,-0.000534,-0.000266,-0.000171,-0.000127,-0.000094
9,0.001988,0.002812,0.003653,0.004981,0.007280,0.011598,0.018225,0.034163,0.071191,0.991095,...,-0.014555,-0.009072,-0.005381,-0.003197,-0.001640,-0.000882,-0.000443,-0.000285,-0.000215,-0.000156


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.002835861844358023,
 0.006704636675595621,
 0.008373002506391725,
 0.0072906418236342585,
 0.005823119257423959,
 0.005480686673234647,
 0.005424786879113852,
 0.005935198295287547,
 0.007373018940118992,
 0.008904592738546047,
 0.011221984978670774,
 0.013709930818012861,
 0.015226092250183232,
 0.016458443898121344,
 0.017596792773611147,
 0.0189300401338649,
 0.019322826828408912,
 0.0219830199435902,
 0.03000715809385701,
 0.0661915169153744,
 0.15901491001975554,
 0.31266648885820314,
 0.2964198682247712]